In [15]:
import pandas as pd
import numpy as pd

import pickle
import joblib 

from pydub import AudioSegment
import numpy as np
from scipy.io.wavfile import read
from scipy.fftpack import fft
import pydub
from scipy import signal
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import speech_recognition as sr

In [22]:
data = pd.read_csv("./Medley-solos-DB/Medley-solos-DB_metadata.csv")
df = pd.DataFrame(data)

In [23]:
df.head()

,subset,instrument,instrument_id,song_id,uuid4
0,test,clarinet,0,0,0e4371ac-1c6a-51ab-fdb7-f8abd5fbf1a3
1,test,clarinet,0,0,33383119-fd64-59c1-f596-d1a23e8a0eff
2,test,clarinet,0,0,b2b7a288-e169-5642-fced-b509c06b11fc
3,test,clarinet,0,0,151b6ee4-313a-58d9-fbcb-bab73e0d426b
4,test,clarinet,0,0,b43999d1-9b5e-557f-f9bc-1b3759659858


In [24]:
test = df["subset"]=="test"
df_test= df[test]
df_test.head()

,subset,instrument,instrument_id,song_id,uuid4
0,test,clarinet,0,0,0e4371ac-1c6a-51ab-fdb7-f8abd5fbf1a3
1,test,clarinet,0,0,33383119-fd64-59c1-f596-d1a23e8a0eff
2,test,clarinet,0,0,b2b7a288-e169-5642-fced-b509c06b11fc
3,test,clarinet,0,0,151b6ee4-313a-58d9-fbcb-bab73e0d426b
4,test,clarinet,0,0,b43999d1-9b5e-557f-f9bc-1b3759659858


In [25]:
train = df["subset"]=="training"
df_train= df[train]
df_train.head()

,subset,instrument,instrument_id,song_id,uuid4
12236,training,clarinet,0,139,163fd2b1-8e98-515a-f501-4742cc6d066f
12237,training,clarinet,0,139,36dcb8eb-da46-5d01-fa9d-0f715c389379
12238,training,clarinet,0,139,9f6d4046-3662-5433-fc44-98c80a428fd8
12239,training,clarinet,0,139,5ef6a0ee-a092-5d49-f64e-a3454e9ab237
12240,training,clarinet,0,139,2048e98b-77b2-5b7d-fe6a-780f399e69fe


In [26]:
validation = df["subset"]=="validation"
df_validation= df[validation]
df_validation.head()

,subset,instrument,instrument_id,song_id,uuid4
18077,validation,clarinet,0,200,a8bfed6b-3005-5059-f46d-4f4c5e672850
18078,validation,clarinet,0,200,cb17632c-ca9f-5b7c-f8fb-58a5f35d48ea
18079,validation,clarinet,0,200,21b3c105-1704-5e7c-fc2f-1b60fdb9d74b
18080,validation,clarinet,0,200,ea3fe67c-23fb-5cc5-fdc8-8c90715af8e1
18081,validation,clarinet,0,200,5aef142f-545d-5ace-f1bb-a96e843a999c


In [27]:
instrument_lst = set(df.instrument.values)  
instrument_lst

{'clarinet',
 'distorted electric guitar',
 'female singer',
 'flute',
 'piano',
 'tenor saxophone',
 'trumpet',
 'violin'}

In [28]:
instrument_lst = list(instrument_lst)

In [29]:
instrument_lst

['distorted electric guitar',
 'tenor saxophone',
 'clarinet',
 'trumpet',
 'flute',
 'female singer',
 'piano',
 'violin']

In [30]:
instruments_train = {}
for inst in instrument_lst:
    instruments_train.update({inst:list(df['uuid4'][df["instrument"]==inst])})


In [31]:
#instruments_train['female singer']

In [32]:
instruments_test = {}
for t in instrument_lst:
    instruments_test.update({t:list(df['uuid4'][df["instrument"]==t])})



In [33]:
instruments_validation = {}
for v in instrument_lst:
    instruments_validation.update({v:list(df['uuid4'][df["instrument"]==v])})



In [34]:
import os
import re

def find(pattern, path):
    ''' 
    Esta función busca el primer fichero que cumpla
    el pattern, en el path que recibe.
    Si lo encuentra devuelve la ruta del fichero.
    '''
    for root, dirs, files in os.walk(path):
        for name in files:
            if bool(re.search(pattern, name)):
                return os.path.join(root, name)
    return None

#find(fr"(.*){identifier}(.*)", './Medley-solos-DB/')

In [35]:
instruments_train_paths = {}
for inst, uuid4_list in instruments_train.items():
    inst_path_list = []
    for uuid4 in uuid4_list:
        path = find(fr"(.*){uuid4}(.*)", './Medley-solos-DB')
        if path:
            inst_path_list.append(path)
    instruments_train_paths[inst] = inst_path_list

In [36]:
# Creamos una copia de instruments_train_paths por si modificamos algo por error,
# para no tener que volver a ejecutar la celda anterior.
instruments_train_paths_copy = instruments_train_paths

In [41]:
def removeEmptyPaths(inst_paths_dict):
    '''
    Esta función limpia los instrumentos cuya lista
    de paths esté vacía y te devuelve una copia del diccionario limpio.
    '''
    inst_paths_dict_cleaned = inst_paths_dict
    for inst, paths in inst_paths_dict.items():
        if not paths:
            inst_paths_dict_cleaned.pop(inst)
        else:
            print(inst)
    return inst_paths_dict_cleaned

In [42]:
instruments_train_paths_copy = removeEmptyPaths(instruments_train_paths_copy)

distorted electric guitar
tenor saxophone
clarinet
trumpet
flute
female singer
piano
violin


In [62]:
def normalizeInstPathsLen(inst_paths_dict):
    '''
    Esta función va a homogeneizar la longitud 
    de las listas de paths que tiene el diccionario
    que recibe, va a crear un nuevo diccionario y 
    lo va a devolver.
    '''
    insts_paths_normalized_len = {}
    min_len = len(list(inst_paths_dict.items())[0][1])
    for inst, paths in inst_paths_dict.items():
        if min_len > len(paths):
            min_len = len(paths)
    for inst, paths in inst_paths_dict.items():
        insts_paths_normalized_len[inst] = paths[:min_len]
        
    return insts_paths_normalized_len

In [64]:
insts_train_paths_normalized = normalizeInstPathsLen(instruments_train_paths_copy)

1854
477
1311
627
3555
1744
6032
5971
477


In [95]:
insts_train_paths_normalized_df = pd.DataFrame.from_dict(insts_train_paths_normalized)
insts_train_paths_normalized_df = insts_train_paths_normalized_df.reset_index()

In [134]:
insts_train_paths_normalized_df.to_csv(r'./normalized_data.csv', index = False)

In [119]:
def samplesList(path_lst):
    '''
    Esta función crea una lista de objetos de AudioSegment
    por cada fichero de la lista que recibe (path_lst),
    después por cada objeto AudioSegment, obtiene la lista
    de samples de cada audio, creando una nueva lista y devolviéndola.
    '''
    audio_lst = []
    audio_samples_lst = []
    for path in path_lst:
        audio = AudioSegment.from_file(path, format='wav')
        audio_lst.append(audio)
    for a in audio_lst:
        audio_samples_lst.append(a.get_array_of_samples())
    
    return audio_samples_lst

def fftList(samples_lst):
    '''
    Esta función recibe una lista de samples de audios,
    hace la transformada de Fourier y devuelve una lista, 
    con toda la lista transformada.
    '''
    fft_lst = []
    for sample in samples_lst:
        four = abs(fft(sample))
        fft_lst.append(four)
    
    return fft_lst

In [102]:
# Cogemos el diccionario y por cada instrumento generamos su lista de samples:

def instSamples(insts_paths):
    '''
    Esta función genera un dict de instrumentos
    y su lista samples a partir del dict de paths 
    que recibe y lo devuelve.
    '''
    inst_samples = {}
    for inst, paths in insts_paths.items():
        inst_samples[inst] = samplesList(paths)
        
    return inst_samples

In [85]:
insts_train_samples = instSamples(insts_train_paths_normalized)

In [116]:
def infoSamples(inst_samples):
    '''
    Esta función halla la longitud máxima, mínima y
    la media de los samples de todos los instrumentos.
    '''
    min_len = len(list(inst_samples.items())[0][1][0])
    max_len = 0
    joined_samples_lst = []
    for inst, samples_lst in inst_samples.items():
        joined_samples_lst = joined_samples_lst + samples_lst
        
    means_lst = []
    for samples in joined_samples_lst:
        samples_len = len(samples)
        means_lst.append(samples_len)
        if min_len > samples_len:
            min_len = samples_len
        if max_len < samples_len:
            max_len = samples_len
    
    print(f'Mean: {sum(means_lst) / len(means_lst)}')
    print(f'Max len: {max_len}')
    print(f'Min len: {min_len}')

In [117]:
infoSamples(insts_train_samples)

Mean: 131072.0
Max len: 131072
Min len: 131072


In [123]:
def instFft(insts_samples):
    '''
    Esta función genera un dict de instrumentos
    y su lista samples a partir del dict de paths 
    que recibe y lo devuelve.
    '''
    inst_ffts = {}
    for inst, samples in insts_samples.items():
        inst_ffts[inst] = fftList(samples)
        
    return inst_ffts

insts_ffts = instFft(insts_train_samples)

In [127]:
def dataFramer(insts_ffts):
    '''
    Esta función toma el diccionario que se le pase como argumento
    y, por cada entrada del diccionario, crea un dataframe y los concatena 
    todos.
    '''
    lst_df = []
    for inst, fft_lst in insts_ffts.items():
        inst_tr = {"transformed": [fft for fft in fft_lst], "type": inst}
        lst_df.append(pd.DataFrame(inst_tr))
    return pd.concat(lst_df).reset_index(drop=True)



In [128]:
df_total = dataFramer(insts_ffts)

In [130]:
df_total.shape

(3816, 2)

In [132]:
df_total.head(30)

,transformed,type
0,"[398163968.0, 511615178.64119005, 691976783.77...",distorted electric guitar
1,"[415727616.0, 428161412.8192643, 434192050.949...",distorted electric guitar
2,"[1990262784.0, 2040929006.2027338, 2128885033....",distorted electric guitar
3,"[1507098624.0, 1525997049.624465, 1295646393.7...",distorted electric guitar
4,"[852787200.0, 850604882.732104, 664008433.7427...",distorted electric guitar
5,"[1723400192.0, 1775458245.3398733, 1808124781....",distorted electric guitar
6,"[634191872.0, 707052860.7757475, 567162736.811...",distorted electric guitar
7,"[50003968.0, 72475598.61380084, 299941823.7584...",distorted electric guitar
8,"[58032128.0, 115911223.5988486, 273369082.4750...",distorted electric guitar
9,"[770408448.0, 790639947.1199154, 961884520.873...",distorted electric guitar


In [131]:
df_total.to_csv(r'./transformed.csv', index = False)